In [1]:
import pandas as pd
import networkx as nx
import numpy as np
import requests
import geopandas as gpd
import osmnx as ox

In [17]:
df_grid = pd.read_excel('./grid.xlsx', index_col=0)
df_substations = pd.read_excel('./substations.xlsx', index_col=0)

pd.set_option('display.max_rows', 100)

In [28]:
df_tielines = pd.read_excel('./jao_static_grid.xlsx', sheet_name='Tielines', skiprows=[0])
df_tielines = df_tielines[[
    'NE_name', 'EIC_Code', 'TSO', 
    'Full_name', 'Full_name.1', 'Voltage_level(kV)', 
    'Resistance_R(Ω)', 'Reactance_X(Ω)', 'Susceptance_B(μS)', 'Length_(km)'
]]
df_tielines = df_tielines.rename(columns={
    'NE_name': 'name', 
    'EIC_Code': 'eic',
    'TSO': 'tso',
    'Voltage_level(kV)': 'voltage',
    'Resistance_R(Ω)': 'resistance',
    'Reactance_X(Ω)': 'reactance',
    'Susceptance_B(μS)': 'susceptance',
    'Length_(km)': 'length',
    'Full_name': 'substation_1',
    'Full_name.1': 'substation_2',
})

tso_zone_map = {
    '50HERTZ': 'DE', 
    'Amprion GmbH': 'DE', 
    'APG': 'AT', 
    'CEPS': 'CZ', 
    'Creos': 'DE', 
    'ELES': 'SI', 
    'ELIA': 'BE',
    'HOPS': 'HR', 
    'MAVIR': 'HU', 
    'PSE': 'PL', 
    'RTE': 'FR', 
    'SEPS': 'SK', 
    'TEL': 'RO', 
    'TENNET NL': 'NL',
    'TENNETGMBH': 'DE', 
    'TRANSNETBW': 'DE'
}

df_tielines['zone'] = df_tielines.apply(lambda row : tso_zone_map[row.tso], axis=1)

In [29]:
def get_from_zone(row):
    if row['eic'][3] == '-':
        return row['eic'][4] + row['eic'][5]
    else:
        return row['zone']
    
def get_to_zone(row):
    if row['eic'][3] == '-':
        return row['eic'][7] + row['eic'][8]
    else:
        return row['zone']

df_tielines['from_zone'] = df_tielines.apply(lambda row: get_from_zone(row), axis=1)
df_tielines['to_zone'] = df_tielines.apply(lambda row: get_to_zone(row), axis=1)

z1 = set(df_tielines['from_zone'].unique())
z1 = z1.union(df_tielines['to_zone'].unique())
zones = list(z1)

relevant_zones = ['DE', 'AT', 'CZ', 'DE', 'SI', 'BE', 'HR', 'HU', 'PL', 'FR', 'SK', 'RO', 'NL', 'D2', 'D4', 'D7', 'D8']

df_tielines = df_tielines[(df_tielines['from_zone'].isin(relevant_zones)) & (df_tielines['to_zone'].isin(relevant_zones))]

In [30]:
substations = df_substations.name.tolist()
substations = list(map(str.strip,substations))
tieline_substations = []

for index, row in df_tielines.iterrows():
    if row['substation_1'].strip() not in tieline_substations:
        tieline_substations.append(row['substation_1'])
    if row['substation_2'].strip() not in tieline_substations:
        tieline_substations.append(row['substation_2'])
        
new_substations = set(list(map(str.lower,tieline_substations))).difference(list(map(str.lower,substations)))
exclude_list = [
    'mukachevo', 'zahidno ukrainska', 'vulcanesti', 'prijedor', # non-fbmc zones
    'mt-saint-martin', # FR, add back later
    'sanem', 'sotel', # LU, add back later
    'nemo convertor station', # uk, non-fbmc
    'vill', 'kempten', 'laufenburg', 'oftringen', 'kasso', 'walgau', # DE, add back later
    'laufenburg', 'oftringen', # swiss, non-fbmc
    'liskowiec', # PL add back later
    'kasso', # DK, non-fbmc
]
new_substations = list(filter(lambda x: x not in exclude_list, new_substations))

substation_zones = []
substation_tso = []

for subst in new_substations:
    matches = df_tielines[df_tielines['substation_1'].apply(str.lower) == subst.strip()]
    print(subst.strip())
    substation_zones.append(matches.iloc[0]['zone'])
    substation_tso.append(matches.iloc[0]['tso'])

df_substations_extra = pd.DataFrame(list(zip(new_substations.copy(), substation_zones.copy(), substation_tso.copy())), columns =['name', 'zone', 'tso'])
df_substations_extra['lat'] = None
df_substations_extra['lon'] = None

bürs


In [21]:
display(df_substations_extra)
df_substations_extra.to_excel("tieline_substations.xlsx")

,name,zone,tso,lat,lon
0,bürs,DE,TRANSNETBW,None,None
1,cikovce,SI,ELES,None,None
2,repiquage bifferdange 220kv,BE,ELIA,None,None
3,meiningen,AT,APG,None,None
4,walgauwerk,AT,APG,None,None
5,monceau,BE,ELIA,None,None
6,prijedor,HR,HOPS,None,None


In [42]:
df_grid = pd.read_excel('./grid.xlsx', index_col=0)
df_substations = pd.read_excel('./substations.xlsx', index_col=0)

non_fbmc_substations = ['mukachevo', 'zahidno ukrainska', 'vulcanesti', 'prijedor', 'nemo convertor station', 'laufenburg', 'oftringen', 'kasso']
df_tielines = df_tielines[(~df_tielines['substation_1'].apply(str.lower).isin(non_fbmc_substations)) & (~df_tielines['substation_2'].apply(str.lower).isin(non_fbmc_substations))]

In [44]:
for index, row in df_tielines.iterrows():
    s1 = df_substations.index[df_substations.name.apply(str.lower).apply(str.strip) == row.substation_1.strip().lower()].tolist()[0]
    s2 = df_substations.index[df_substations.name.apply(str.lower).apply(str.strip) == row.substation_2.strip().lower()].tolist()[0]
    df_tielines.loc[index, 'substation_1'] = s1
    df_tielines.loc[index, 'substation_2'] = s2

In [48]:
#df_tielines = df_tielines.drop(columns=['from_zone', 'to_zone'])
#df_tielines['tieline'] = True 
display(df_tielines)
df_tielines.to_excel("tielines_final.xlsx")

,name,eic,tso,substation_1,substation_2,voltage,resistance,reactance,susceptance,length,zone,tieline
0,Redwitz - Remptendorf 413,11T-D2-D8-00050X,50HERTZ,496,598,400,0.731,7.216,126.60600,18.40,DE,True
1,Redwitz - Remptendorf 414,11T-D2-D8-00051V,50HERTZ,496,598,400,0.731,7.216,126.60600,18.40,DE,True
2,Röhrsdorf - Hradec 445,10T-CZ-DE-00002U,50HERTZ,570,778,400,1.230,10.945,174.82960,51.90,DE,True
3,Röhrsdorf - Hradec 446,10T-CZ-DE-00003S,50HERTZ,570,778,400,1.230,10.945,174.82960,51.90,DE,True
4,Mecklar - Vieselbach 449-1,11T-D2-D8-000302,50HERTZ,1470,657,400,1.774,16.325,267.06700,87.10,DE,True
...,...,...,...,...,...,...,...,...,...,...,...,...
240,Buers - Westtirol rt (422),10T-AT-DE-000053,TRANSNETBW,870,794,400,0.990,14.270,168.70350,91.40,DE,True
241,Buers - Westtirol ws (421),10T-AT-DE-000061,TRANSNETBW,870,794,220,1.000,14.365,170.11730,92.00,DE,True
242,Buers - Y-Werben,11T-D4-D7-00001W,TRANSNETBW,870,987,220,0.595,90.800,70.41881,47.00,DE,True
243,Eichstetten - Muhlbach rt (Ill),10T-DE-FR-00002G,TRANSNETBW,543,236,400,0.380,4.100,70.34027,17.38,DE,True
